In [34]:
import os
import pandas as pd
import datetime as dt
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

In [35]:
fn = "raw_2021-07-25.csv"
date_str = fn.split(".")[0].split("_")[1]
df = pd.read_csv(os.path.join("..", "..", "logs", fn))
df.tail()

,time,ver,charge_state,I_L1,I_L2,I_L3,Temp,V_L1,V_L2,V_L3,...,E_cyc_lb,E_hb,E_lb,I_max,I_min,watchdog,standby,remote_lock,max_I_cmd,FailSafe_I
6685,10:23:20,264,2,0,0,0,328,237,235,239,...,12088,2,61695,10,6,15000,4,1,100,10
6686,10:23:26,264,2,0,0,0,325,236,235,238,...,12088,2,61695,10,6,15000,4,1,100,10
6687,10:23:31,264,2,0,0,0,328,235,234,238,...,12088,2,61695,10,6,15000,4,1,100,10
6688,10:23:36,264,2,0,0,0,328,235,234,238,...,12088,2,61695,10,6,15000,4,1,100,10
6689,10:23:41,264,2,0,0,0,328,235,234,238,...,12088,2,61695,10,6,15000,4,1,100,10


In [2]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'
db = SQLAlchemy(app)

C:\ProgramData\Anaconda3\envs\flask_env\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [50]:
class WalliStat(db.Model):
    dt = db.Column(db.DateTime, nullable=False, primary_key=True)
    Temp = db.Column(db.Integer, nullable=False, unique=False)
    Power = db.Column(db.Integer, nullable=False, unique=False)
    
    def __repr__(self):
        return f"WalliStat({self.dt}: {self.Temp}°C, {self.Power}W)"

In [51]:
db.create_all()

In [53]:
for index, row in df.tail().iterrows():
    date = pd.to_datetime(date_str + " " + row.time).to_pydatetime()
    ws = WalliStat(dt=date, Temp=row.Temp, Power=row.P)
    db.session.add(ws)
    print(ws)

WalliStat(2021-07-25 10:23:20: 328°C, 0W)
WalliStat(2021-07-25 10:23:26: 325°C, 0W)
WalliStat(2021-07-25 10:23:31: 328°C, 0W)
WalliStat(2021-07-25 10:23:36: 328°C, 0W)
WalliStat(2021-07-25 10:23:41: 328°C, 0W)


In [54]:
db.session.commit()

In [58]:
WalliStat.query.all()

[WalliStat(2021-07-25 10:23:20: 328°C, 0W),
 WalliStat(2021-07-25 10:23:26: 325°C, 0W),
 WalliStat(2021-07-25 10:23:31: 328°C, 0W),
 WalliStat(2021-07-25 10:23:36: 328°C, 0W),
 WalliStat(2021-07-25 10:23:41: 328°C, 0W)]

In [66]:
WalliStat.query.filter_by(Temp = 328).all()

[WalliStat(2021-07-25 10:23:20: 328°C, 0W),
 WalliStat(2021-07-25 10:23:31: 328°C, 0W),
 WalliStat(2021-07-25 10:23:36: 328°C, 0W),
 WalliStat(2021-07-25 10:23:41: 328°C, 0W)]